In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import gc
import pickle
from tqdm.notebook import tqdm

In [2]:
import itertools

In [3]:
def load_file(file):

    r = open(file,'rb')
    x = pickle.load(r)
    r.close()

    return x

In [4]:
def save_file(x,file):

    r = open(file,'wb')
    pickle.dump(x,r)

In [5]:
#import Theorem data
list_all = pd.read_excel('F:\\dataset\\mpeuni\\Theorems\\new\\nodelist_all.xlsx', header = 0)
adjacency_listT = pd.read_excel('F:\\dataset\\mpeuni\\Theorems\\new\\adjacency_list_Theorems_and_Axioms_only.xlsx', header = 0)
F = nx.from_pandas_edgelist(adjacency_listT, 'Theorem_id', 'Ref_id', create_using=nx.DiGraph())
F2 = F.reverse(copy=True)

In [6]:
#import other data
name1 = 'XXX.pkl'
Fdata = load_file(name1)
F2 = F.reverse(copy=True)

In [7]:
nodelist = list(F)
topological_generationsTGM = [sorted(generation) for generation in nx.topological_generations(F2)]
generation = []
for k in range(len(nodelist)):
    pindex = []
    for p in range(len(topological_generationsTGM)):
        pindex.append(nodelist[k] in topological_generationsTGM[p])
    p1 = pindex.index(True)
    generation.append(p1)

In [8]:
Disruption = []
njcount = []
nicount = []
nkcount = []

In [9]:
for i in tqdm(range(len(nodelist))):
    p1 = generation[i]
    node_index = nodelist.index
    neighbors_F2 = list(F2.neighbors(nodelist[i]))
    neighbors_F = list(F.neighbors(nodelist[i]))

    neighbor1 = [neighbor for neighbor in neighbors_F2 if generation[node_index(neighbor)] < p1 + 11]

    neighbor3_1 = list(itertools.chain.from_iterable(list(F2.neighbors(neighbor)) for neighbor in neighbors_F))
    nj = set(neighbor3_1).intersection(neighbor1)
    ni = set(neighbor1).difference(nj)
    nicount.append(len(ni))
    njcount.append(len(nj))

    nk = [nkor_node for nkor_node in set(neighbor3_1).difference(nj) if p1 < generation[node_index(nkor_node)] < p1 + 11]
    nkcount.append(len(nk))
    
    total_neighbors = len(ni) + len(nj) + len(nk)
    
    if total_neighbors==0:
        Disruptionx = None
    else:
        Disruptionx = (len(ni)-len(nj))/total_neighbors
    Disruption.append(Disruptionx)

  0%|          | 0/27400 [00:00<?, ?it/s]

In [10]:
Disruptiondata = pd.DataFrame({'Generation':generation, 'Disruption':Disruption, 'nj':njcount, 'ni':nicount, 'nk':nkcount}, index=nodelist)

In [11]:
def degree_distribution(F):
    degree = pd.DataFrame(nx.degree(F))
    degree = degree.sort_values(by=[1], ascending=False)
    citypdf_sum2 = pd.DataFrame(degree[1].value_counts())
    citypdf_sum2['f'] = citypdf_sum2.index
    citypdf_sum2['频率'] = citypdf_sum2[1]/citypdf_sum2[1].sum()
    citypdf_sum2 = citypdf_sum2.sort_values(by=['f'], ascending=True)
    ccdf_sum2 = np.zeros(len(citypdf_sum2))
    for i in range(len(ccdf_sum2)):
        ccdf_sum2[i] = 1-np.sum(citypdf_sum2.values[0:i,2])
    citypdf_sum2['ccdf'] = ccdf_sum2
    gdo = pd.DataFrame(F.out_degree())
    gdo = gdo.sort_values(by=[1], ascending=False)
    citypdf_sum = pd.DataFrame(gdo[1].value_counts())
    citypdf_sum['f'] = citypdf_sum.index
    citypdf_sum['频率'] = citypdf_sum[1]/citypdf_sum[1].sum()
    citypdf_sum = citypdf_sum.sort_values(by=['f'], ascending=True)
    ccdf_sum = np.zeros(len(citypdf_sum))
    for i in range(len(ccdf_sum)):
        ccdf_sum[i] = 1-np.sum(citypdf_sum.values[0:i,2])
    citypdf_sum['ccdf'] = ccdf_sum
    gdi = pd.DataFrame(F.in_degree())
    gdi = gdi.sort_values(by=[1], ascending=False)
    citypdf_sum1 = pd.DataFrame(gdi[1].value_counts())
    citypdf_sum1['f'] = citypdf_sum1.index
    citypdf_sum1['频率'] = citypdf_sum1[1]/citypdf_sum1[1].sum()
    citypdf_sum1 = citypdf_sum1.sort_values(by=['f'], ascending=True)
    ccdf_sum1 = np.zeros(len(citypdf_sum1))
    for i in range(len(ccdf_sum1)):
        ccdf_sum1[i] = 1-np.sum(citypdf_sum1.values[0:i,2])
    citypdf_sum1['ccdf'] = ccdf_sum1
    degree.set_index([0], inplace=True)
    gdo.set_index([0], inplace=True)
    gdi.set_index([0], inplace=True)
    degree_all = pd.concat([degree, gdo, gdi], axis=1)
    degree_all.columns = ['Degree','Out-degree','In-degree']
    return degree_all, citypdf_sum2, citypdf_sum, citypdf_sum1

In [12]:
FTGMdegree_all, FTGMdegreepdf, FTGMoutdegreepdf, FTGMindegreepdf = degree_distribution(F)

In [13]:
Disruptiondata = pd.merge(Disruptiondata, FTGMdegree_all, left_index=True, right_index=True)

In [14]:
Disruptiondata['Citation'] = Disruptiondata['ni'] + Disruptiondata['nj']

In [15]:
Disruptiondata['Clustering'] = None

In [16]:
name2 = 'Disruptiondata10XXX.pkl'

In [17]:
save_file(Disruptiondata,name2)